In [1]:
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 전체 data 수집하기

기본값

In [2]:
api_limit = 10000  #max : 1000000
baseurl = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
servicekey = 'Ex36SLserq7pqUqFo2O9VRpkce6bHGi2zwKRgdo%2BBLxi77pw2lgEqLR67DQkh3x5q%2Bhn5hLN8BKk6dJgPFjTWw%3D%3D'
dealym = '201901'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}

parsed_data = []

[get_dealym] 시작일부터 종료일까지의 날짜

In [3]:
def get_dealym(start_ym, end_ym) :
    res = []
    start = datetime.strptime(start_ym, '%Y%m')
    end = datetime.strptime(end_ym, '%Y%m')
    diff = (end.year - start.year) * 12 + (end.month  - start.month) + 1
    for i in range(diff) :
        res.append(datetime.strftime(start + relativedelta(months = i), '%Y%m')) # 기준날짜에 대한 연산이 필요할때 relativedelta
    return res

url 가져오기

In [4]:
def get_url(pageno, lawd_cd, deal_ym) :
    params = {'ServiceKey' : servicekey, 
              'pageNo' : pageno, 
              'numOfRows' : '10', 
              'LAWD_CD' : lawd_cd, 
              'DEAL_YMD' : deal_ym }

    url = baseurl + '?' + '&'.join(['{}={}'.format(k,v) for k, v in params.items()])
    return url

[get_data] request로 xml에서 필요한 데이터 가져오기

In [5]:
def get_data(url) :
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml-xml')
    items = soup.findAll('item')
    return items

[get_data_firstpg] 전체 결과에 대한 page수 확인

In [6]:
def get_data_firstpg(url) :
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml-xml')
    items = soup.findAll('item')
    totalcount = int(soup.find('totalCount').text)
    return items, 1 + int(totalcount/10)

[parse_text] if문으로 null값 호출시 error해결

In [7]:
def parse_text(soup_object) : 
    if soup_object :
        return soup_object.text
    else :
        return ''

[parse_item] 데이터 파싱

In [8]:
def parse_item(item, deal_ym) :
    amount = parse_text(item.find('거래금액')).strip().replace(',', '')
    
    road_aptcode = parse_text(item.find('도로명건물본번호코드'))
    road_aptcode2 = parse_text(item.find('도로명건물부번호코드'))
    road_addr = item.find('도로명')
    if item.find('도로명') == '' :
        road_addr = ''
    else :
        road_addr = parse_text(item.find('도로명')).strip()

    if road_addr == '' :
        road_addr = ''
    elif (road_aptcode2 == '00000') or (road_aptcode2 == ''):
        road_addr = "{} {}".format(road_addr, int(road_aptcode))
    else :
        road_addr = "{} {}-{}".format(road_addr, int(road_aptcode), int(road_aptcode2))

    apt_dong = parse_text(item.find('법정동'))
    apt_lawd = parse_text(item.find('법정동시군구코드'))    
    apt_name = parse_text(item.find('아파트'))
    apt_year = parse_text(item.find('건축년도'))
    apt_area = parse_text(item.find('전용면적'))
    apt_floor = parse_text(item.find('층'))
    deal_y = int(str(deal_ym)[:4])
    deal_m = parse_text(item.find('월'))
    deal_d = parse_text(item.find('일'))
    deal_ymd = "{}{:02d}{:02d}".format(deal_y, int(deal_m), int(deal_d))
    deal_datetime = datetime.strptime(deal_ymd, '%Y%m%d')
    
    res = {'아파트' : apt_name, '건축년도' : int(apt_year), '전용면적' : float(apt_area), '층' : int(apt_floor),
           '거래금액' : int(amount), '주소' : road_addr, '법정동' : apt_dong, '법정동코드' : apt_lawd, 
           '거래일자' : deal_datetime, '거래일_연도' : deal_y, '거래일_월' : deal_m, '거래일_일' : deal_d,
          }
    return res

[run] 실행함수

In [9]:
def run(pageno, lawd_cd, deal_ym) :
    url = get_url(pageno, lawd_cd, deal_ym)
    print(url)
    items = get_data(url)
    if items :
        parsed_data.extend([parse_item(x, deal_ym) for x in items])
        return True
    else :
        return False

[save_data] 데이터 저장

In [10]:
def save_data(parsed_data) :
    df = pd.DataFrame(parsed_data)
    df.to_csv("data/apt_deal_2019_01_12.csv" ,index=True, encoding='utf-8-sig')

[main] 분석

In [11]:
%%time

if __name__ == '__main__' : #필수적인 것(그냥외워두삼)
    start_ym = '201901'
    end_ym = '201912'

    ls_dealym = get_dealym(start_ym, end_ym) #날짜리스트
    lawd_csv_path = 'data/apt_dongcode.csv' #법정동코드 파일 위치
    df_lawd = pd.read_csv(lawd_csv_path, encoding='utf-8-sig')
    ls_lawd = list(df_lawd['법정동코드']) #법정동리스트

    dealym = ls_dealym[0]
    pg = 1
    lawd = ls_lawd[0]

    if dealym != ls_dealym[0] :
        idx_ym = ls_dealym.index(dealym)
        ls_dealym = ls_dealym[idx_ym:]
    if lawd != ls_lawd[0] :
        idx_lawd = ls_lawd.index(lawd)
        ls_lawd = ls_lawd[idx_lawd:]

    count = 0

    for lawdcode in ls_lawd :
        for ym in ls_dealym :
            while(run(pg, lawdcode, ym)) :
                count += 1
                pg += 1
                time.sleep(0.1)
                if count >= api_limit :
                    pg_next = pg+1
                    break
            
            if count >= api_limit :
                pg_next = pg
                break

            count += 1
            pg = 1

        else :
            continue
        break

    '''for lawdcode in ls_lawd :
        for ym in ls_dealym :
            url_firstpg = get_url(1, lawdcode, ym)
            items, pg_end = get_data_firstpg(url_firstpg)
            print(url_firstpg)
            count += 1
            if items :
                parsed_data.extend([parse_item(x, ym) for x in items])
            if pg_end > 1 :
                for pg in range(2, pg_end+1) :
                    run(pg, lawdcode, ym)
                    count += 1
                    time.sleep(1)
                    if count >= api_limit :
                        pg_next = pg+1 if pg != pg_end else 1
                        break
            
            if count >= api_limit :
                pg_next = 1
                break
            
        else :
            continue
        break'''


    save_data(parsed_data) #데이터저장
    #마지막저장기록 
    now = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M')
    with open('openapi_log.txt', 'a+') as f :
        f.write("크롤링날짜\t법정동코드\t거래일\t페이지\n")
        f.write("{}\t{}\t{}\t".format(now, lawdcode, ym))
        f.close()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?ServiceKey=Ex36SLserq7pqUqFo2O9VRpkce6bHGi2zwKRgdo%2BBLxi77pw2lgEqLR67DQkh3x5q%2Bhn5hLN8BKk6dJgPFjTWw%3D%3D&pageNo=3&numOfRows=10&LAWD_CD=11530&DEAL_YMD=201912
http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?ServiceKey=Ex36SLserq7pqUqFo2O9VRpkce6bHGi2zwKRgdo%2BBLxi77pw2lgEqLR67DQkh3x5q%2Bhn5hLN8BKk6dJgPFjTWw%3D%3D&pageNo=4&numOfRows=10&LAWD_CD=11530&DEAL_YMD=201912
http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?ServiceKey=Ex36SLserq7pqUqFo2O9VRpkce6bHGi2zwKRgdo%2BBLxi77pw2lgEqLR67DQkh3x5q%2Bhn5hLN8BKk6dJgPFjTWw%3D%3D&pageNo=5&numOfRows=10&LAWD_CD=11530&DEAL_YMD=201912
http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?ServiceKey=Ex36SLserq7pqUqFo2O9VRpkce6bHGi2zwKRgdo%2BBLxi77pw2lgEqLR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')